- fastapi는 무엇이고, 언제 사용을 하는지등 기본적인 이론도 같이 공부하면 좋을 것 같습니다.
  - FastAPI , uvicorn , ...

### 01. 필요한 라이브러리 설치

In [1]:
!pip install FastAPI -q
!pip install uvicorn -q
!pip install torch
!pip install python-multipart
!pip install fastapi nest-asyncio pyngrok uvicorn


[notice] A new release of pip is available: 24.1.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 24.1.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.1.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.1.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.1.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


### 02. FastAPI 실행

- 학습때 설치했던 yolov5 폴더가 있는 경로로 이동해주세요.

In [12]:
import os
os.getcwd()

'A:\\commento\\2_model'

In [13]:
cd A:\commento\2_model


A:\commento\2_model


- 학습한 모델을 로드해줍니다.

In [5]:
from ultralytics import YOLO
import os
import torch
import uvicorn
import nest_asyncio

from PIL import Image
from io import BytesIO
from pyngrok import ngrok
from torchvision import transforms
from fastapi import FastAPI, File, UploadFile, Request
from fastapi.middleware.cors import CORSMiddleware
import time


# FastAPI 앱 생성
app = FastAPI()

# YOLOv5 모델 로드
model = YOLO(r"A:\commento\2_model\runs\detect\train\weights\best.pt")
model.conf = 0.5  # 신뢰도 임계값 설정

# CORS 허용 설정
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # 모든 도메인 허용 (개발용)
    allow_credentials=True,
    allow_methods=["*"],  # 모든 HTTP 메소드 허용
    allow_headers=["*"],  # 모든 HTTP 헤더 허용
)

'''
# ngrok 인증 토큰 설정
authtoken = "2mYsZHJc2N48scwLSCgcje51ehv_86wfXdYxtFkQAPrryCWy" # 자신의 인증 토큰으로 변경하세요
ngrok.set_auth_token(authtoken)
'''

'\n# ngrok 인증 토큰 설정\nauthtoken = "2mYsZHJc2N48scwLSCgcje51ehv_86wfXdYxtFkQAPrryCWy" # 자신의 인증 토큰으로 변경하세요\nngrok.set_auth_token(authtoken)\n'

### fastapi 실행 함수 구현
- Input : 이미지 경로
- Output : 자유롭게 아이디어를 내서 결과를 출력시켜주세요.
  - 차량 정체 문제를 파악하기 위한 값으로 출력해주세요.
  - 예를 들어, 차량 개수 파악 / 차량 전체 면적 또는 비율 등 다양한 아이디어를 생각해서 구현해주세요.

#### 내가 처음에 짠 코드

In [42]:
from PIL import ImageDraw
from fastapi.responses import FileResponse
from fastapi.responses import StreamingResponse

app = FastAPI()

# 전역 변수로 이미지 데이터를 저장할 공간
image_with_boxes_bytes = None

# 바운딩 박스를 그리는 함수
def draw_boxes(image, detections_list):
    draw = ImageDraw.Draw(image)
    for detection in detections_list:
        # 바운딩 박스 그리기
        draw.rectangle([(detection["x1"], detection["y1"]),
                        (detection["x2"], detection["y2"])], outline="red", width=3)
        # 클래스 라벨 추가
        draw.text((detection["x1"], detection["y1"] - 10), f"Class: {detection['class']}", fill="red")
    return image

# API에서 추론 후 시각화하는 코드
@app.post('/detect')
async def detect_api(file: UploadFile = File(...)):

    # 전역 변수를 업데이트할 수 있도록 global 선언
    global image_with_boxes_bytes

    # 비동기적으로 파일 읽기
    image_bytes = await file.read()

    try:
        # RGB 형태로 이미지 열기
        image = Image.open(BytesIO(image_bytes)).convert("RGB")

        # YOLOv5 모델 추론
        results = model(image)

        # 결과 처리
        detections_list = []
        df = results.pandas().xyxy[0]

        for index, row in df.iterrows():
            detections_list.append({
                "x1": float(row['xmin']),
                "y1": float(row['ymin']),
                "x2": float(row['xmax']),
                "y2": float(row['ymax']),
                "confidence": float(row['confidence']),
                "class": int(row['class'])
            })

        # 감지된 바운딩 박스(차량)의 개수
        vehicle_count = len(detections_list)

        # 바운딩 박스를 그린 이미지 반환
        image_with_boxes = draw_boxes(image, detections_list)
        
        # 메모리에서 이미지를 바이너리 형태로 저장
        image_with_boxes_bytes = BytesIO()
        image_with_boxes.save(image_with_boxes_bytes, format='JPEG')
        image_with_boxes_bytes.seek(0)

        # 이미지 파일을 /image로 요청할 수 있도록 준비
        return {
            "vehicle_count": vehicle_count,
            "detections": detections_list,
            "image_url": "/image"
        }

    except Exception as e:
        return {"error": "이미지 처리 실패", "details": str(e)}

# 이미지 반환 경로
@app.get("/image")
async def get_image():
    global image_with_boxes_bytes

    if image_with_boxes_bytes is None:
        return {"error": "이미지 파일이 존재하지 않습니다. 먼저 /detect API를 호출하세요."}
    
    # 메모리에 저장된 이미지를 반환
    image_with_boxes_bytes.seek(0)  # 이미지를 읽기 위해 처음으로 되돌림
    return StreamingResponse(image_with_boxes_bytes, media_type="image/jpeg")

# ngrok 터널 설정
port = 8000
public_url = ngrok.connect(port)
print(f"Public URL: {public_url}")

# colab에서 비동기 실행 문제 해결
nest_asyncio.apply()

# FastAPI 서버 실행
uvicorn.run(app, host="0.0.0.0", port=port)



Public URL: NgrokTunnel: "https://4f87-211-178-82-102.ngrok-free.app" -> "http://localhost:8000"


INFO:     Started server process [9136]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     211.178.82.102:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     211.178.82.102:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     211.178.82.102:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     211.178.82.102:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     211.178.82.102:0 - "POST /detect HTTP/1.1" 200 OK
INFO:     211.178.82.102:0 - "GET /image HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [9136]


taskkill /F /IM ngrok.exe

#### 멘토님 피드백 반영한 코드

In [7]:
from fastapi import FastAPI, UploadFile, File
from fastapi.responses import StreamingResponse
from PIL import Image, ImageDraw
from io import BytesIO
import uvicorn
import nest_asyncio
import torch
import pandas as pd
# from pyngrok import ngrok

app = FastAPI()

'''전역 상태 관리 (이미지 바이트 데이터를 저장)
- 이미지를 보여주기 위해 전역변수를 사용하여 저장하는 부분에서 좋은 접근이었던 것 같습니다!
- 하지만, 전역 변수를 최소화하고 필요한 경우는 클래스나 모듈 상태로 관리하는것이 추후 코드의 규모가 커질 경우 관리하기 용이합니다.

그리고, 결과 이미지를 화면에 출력할때도 여러 방법이 있을 것 같습니다. 
- 첫번째로는 추론된 결과 이미지를 물리적으로 저장해서 불러오는 방식
- 두번째로는 추론된 결과 이미지를 전역변수를 사용하여 메모리상에 저장 후 불러오는 방식 
실무에서는 첫번째 방식으로, 추롣된 결과를 내부 DB에 쌓아 불러오는 방식으로 많이 사용되곤 합니다!
'''
class ImageData:
    def __init__(self):
        self.image_with_boxes_bytes = None

image_data = ImageData()

# 바운딩 박스를 그리는 함수
def draw_boxes(image, detections_list):
    draw = ImageDraw.Draw(image)
    for detection in detections_list:
        # 바운딩 박스 그리기
        draw.rectangle([(detection["x1"], detection["y1"]),
                        (detection["x2"], detection["y2"])], outline="red", width=3)
        # 클래스 라벨 추가
        draw.text((detection["x1"], detection["y1"] - 10), f"Class: {detection['class']}", fill="red")
    return image

'''YOLO 모델 추론 함수
- YOLO 모델 추론과 바운딩 박스를 그리는 부분을 별도의 함수(yolo_inference)로 분리하면서 함수의 기능을 좀 더 명확하게 구분 가능합니다.
- YOLOv5 기반 코드
def yolo_inference(image):
    results = model(image)
    df = results.pandas().xyxy[0]
    
    detections_list = [
        {
            "x1": float(row['xmin']),
            "y1": float(row['ymin']),
            "x2": float(row['xmax']),
            "y2": float(row['ymax']),
            "confidence": float(row['confidence']),
            "class": int(row['class'])
        }
        for _, row in df.iterrows()
    ]
    
    return detections_list, len(detections_list)
'''
# YOLOv8 모델 추론 함수 수정
# YOLOv8에서는 pandas() 메서드가 지원되지 않고, 
# 대신 **results.boxes**를 사용하여 바운딩 박스의 좌표 및 기타 정보를 추출해야 함 / 리스트 형태
def yolo_inference(image):
    # YOLOv8 모델 추론
    results = model(image)
    
    # 결과를 리스트로 처리
    detections_list = []
    for result in results:
        for box in result.boxes:
            detections_list.append({
                "x1": float(box.xyxy[0][0]),  # 바운딩 박스 좌상단 x 좌표
                "y1": float(box.xyxy[0][1]),  # 바운딩 박스 좌상단 y 좌표
                "x2": float(box.xyxy[0][2]),  # 바운딩 박스 우하단 x 좌표
                "y2": float(box.xyxy[0][3]),  # 바운딩 박스 우하단 y 좌표
                "confidence": float(box.conf[0]),  # 신뢰도
                "class": int(box.cls[0])  # 클래스 라벨
            })
    
    vehicle_count = len(detections_list)
    return detections_list, vehicle_count

# 바운딩 박스 면적을 계산하는 함수
def calculate_vehicle_area(detections_list):
    vehicle_area = 0
    for detection in detections_list:
        # 바운딩 박스의 넓이 계산 (width * height)
        width = detection["x2"] - detection["x1"]
        height = detection["y2"] - detection["y1"]
        vehicle_area += width * height
    return vehicle_area

# 감지된 이미지 반환 및 바운딩 박스 그리기 + 도로 면적 대비 차량 면적 비율 계산
@app.post('/detect')
async def detect_api(file: UploadFile = File(...)):
    try:
        # 이미지 파일을 비동기적으로 읽기
        image_bytes = await file.read()
        image = Image.open(BytesIO(image_bytes)).convert("RGB")
        
        # YOLOv5 모델 추론
        detections_list, vehicle_count = yolo_inference(image)

        # 바운딩 박스가 그려진 이미지 생성
        image_with_boxes = draw_boxes(image, detections_list)
        
        # 차량 면적 계산
        vehicle_area = calculate_vehicle_area(detections_list)
        
        # 이미지의 전체 면적 (도로 면적) 계산
        total_area = image.width * image.height
        
        # 차량 면적 비율 계산
        congestion_ratio = vehicle_area / total_area

        # 이미지 데이터를 메모리에 저장
        image_data.image_with_boxes_bytes = BytesIO()
        image_with_boxes.save(image_data.image_with_boxes_bytes, format='JPEG')
        image_data.image_with_boxes_bytes.seek(0)

        # 차량 개수 및 도로 정체 정보 반환
        return {
            "vehicle_count": vehicle_count,
            "detections": detections_list,
            "congestion_ratio": congestion_ratio,  # 차량 면적 비율 추가
            "image_url": "/image"
        }

    except Exception as e:
        return {"error": "이미지 처리 실패", "details": str(e)}


# 바운딩 박스를 그린 이미지 반환
@app.get("/image")
async def get_image():
    if image_data.image_with_boxes_bytes is None:
        return {"error": "이미지 파일이 존재하지 않습니다. 먼저 /detect API를 호출하세요."}
    
    image_data.image_with_boxes_bytes.seek(0)  # 이미지 포인터를 처음으로 되돌림
    return StreamingResponse(image_data.image_with_boxes_bytes, media_type="image/jpeg")

@app.middleware("http")
async def add_process_time_header(request: Request, call_next):
    start_time = time.time()
    response = await call_next(request)
    process_time = time.time() - start_time
    response.headers["X-Response-Time"] = str(process_time)
    return response



# Colab 또는 Jupyter Notebook에서 비동기 실행 문제 해결
nest_asyncio.apply()

# FastAPI 서버 실행
if __name__ == "__main__":
    uvicorn.run(app, host="localhost", port=8000)

# 주소 뒤에 /docs 붙이기


INFO:     Started server process [23976]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://localhost:8000 (Press CTRL+C to quit)


INFO:     ::1:54631 - "GET /docs HTTP/1.1" 200 OK
INFO:     ::1:54631 - "GET /openapi.json HTTP/1.1" 200 OK

0: 128x224 (no detections), 17.0ms
Speed: 2.0ms preprocess, 17.0ms inference, 2.0ms postprocess per image at shape (1, 3, 128, 224)
INFO:     ::1:54636 - "POST /detect HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [23976]
